In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [33]:
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()))
url='https://infogob.jne.gob.pe/Eleccion'
driver.get(url)
time.sleep(5)

In [34]:
###Tipo de proceso
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()

In [35]:
##Dar click para que aparezca la lista de opciones
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()

##Obtener la lista
##Grupo completo
lista_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')

##Elemento
opciones_lista_eleccion = [ x for x in lista_eleccion.find_elements(By.TAG_NAME, "div") ]

##Crear un diccionario
dict_opciones_lista_eleccion = { option.text : option for option in opciones_lista_eleccion }

In [36]:
df_resultados = pd.DataFrame(columns=['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS'])

# Fifth, iterate for all the elements in the dictionary created based on list of election years
for elemento_txt in list(dict_opciones_lista_eleccion.keys())[1:]:
    elemento = dict_opciones_lista_eleccion[elemento_txt]
    elemento.click() # click on the second element, the first is "SELECCION" so we don't need it

    # click on the button to see results
    boton_ver_datos = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.ID, 'btnVerDatos'))
    ).click()
    time.sleep(5)
    
    # choose the tab 'CANDIDATOS Y RESULTADOS'
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]').click()
    time.sleep(3)

    #tablas

    table_1= driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]/table')
    table1_html=table_1.get_attribute("outerHTML")
    table1= pd.read_html(table1_html)[0]

    table1['ELECCIONES'] = elemento_txt
    new_order1 = ['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']
    df1 = table1[new_order1]
    
    # concat results in df_results
    df_resultados = pd.concat([df_resultados, df1], ignore_index=True)
    
    # back twice to start again
    driver.back()
    time.sleep(3)

    driver.back()
    time.sleep(3)
    
    # click on the list in order to chosee the next
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()
    time.sleep(3)



C:\Users\kchauca\AppData\Local\Temp\ipykernel_1544\3288455072.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table1= pd.read_html(table1_html)[0]
C:\Users\kchauca\AppData\Local\Temp\ipykernel_1544\3288455072.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table1= pd.read_html(table1_html)[0]
C:\Users\kchauca\AppData\Local\Temp\ipykernel_1544\3288455072.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table1= pd.read_html(table1_html)[0]
C:\Users\kchauca\AppData\Local\Temp\ipykernel_1544\3288455072.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future ver

In [37]:
df_resultados

,ELECCIONES,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS,ELECCIÓN
0,NaN,PARTIDO POLITICO NACIONAL PERU LIBRE,8836380,PRESIDENCIAL 2021 - 2DA VUELTA
1,NaN,FUERZA POPULAR,8792117,PRESIDENCIAL 2021 - 2DA VUELTA
2,NaN,VOTOS EN BLANCO,121489,PRESIDENCIAL 2021 - 2DA VUELTA
3,NaN,VOTOS NULOS,1106816,PRESIDENCIAL 2021 - 2DA VUELTA
4,NaN,PARTIDO POLITICO NACIONAL PERU LIBRE,2724752,PRESIDENCIAL 2021
...,...,...,...,...
152,NaN,PARTIDO REPUBLICANO,30803,PRESIDENCIAL 1936
153,NaN,UNION REVOLUCIONARIA,152149,PRESIDENCIAL 1931
154,NaN,PARTIDO APRISTA PERUANO,106088,PRESIDENCIAL 1931
155,NaN,PARTIDO DESCENTRALISTA,21950,PRESIDENCIAL 1931


In [ ]:
df_resultados.to_excel('resultados_grupo1.xlsx', index=False)